# Adama example for DesignSafe-CI

This is an example of building an Adama service.  We use the Haiti Earthquake Database and we construct a couple of web services from the data hosted at https://nees.org/dataview/spreadsheet/haiti.

## Setting up

The code for these services is in the directory `demo`:

In [ ]:
cd demo

We will construct two web services that will return the data in JSON format:

- `haiti`: will allow to query the database by building,
- `haiti_images`: will allow to retrieve the set of images for each building.

Each Adama service consists in two pieces of information:

- the metadata that describes the service,
- and the actual code.

This is an example of the metadata for the `haiti` service:

In [ ]:
# %load services/haiti/metadata.yml
---
name: haiti
version: 0.1
type: query
main_module: services.haiti.main.py
description: |
  Wrap of the Haiti Earthquake database. On January 12, 2010 an
  earthquake ravaged Haiti (USGS, 2010). In the aftermath of the
  earthquake, a team of professors and students from Purdue
  University, the University of Washington, and the University of
  Kansas went to Haiti. The goal of the team was to evaluate the
  Priority Index proposed by Hassan and Sözen (1997).
icon: nees.png

authors:
  - name: Walter Moreira
    email: wmoreira@tacc.utexas.edu
    sponsor_organization_name: "University of Texas"
    sponsor_uri: "utexas.edu"

endpoints:
  /search:
    description: Query database by building id
    parameters:
      - name: building
        description: Identifier of the building
        type: string
        required: true
        default: A001
  /list:
    description: List all rows in the database
    parameters: []

sources:
  - title: "The Haiti Earthquake Database"
    description: |
      The goal of the team was to evaluate the Priority Index proposed
      by Hassan and Sözen (1997).
    last_modified: "2010-12-21T00:00:00"
    sponsor_organization_name: "Purdue University"
    sponsor_uri: "http://www.purdue.edu/"
    provider_name: "Nathaniel Sedra"
    provider_email: "sedra@example.com"
    uri: "https://nees.org/dataview/spreadsheet/haiti"

The code for the service looks very simple:

In [ ]:
%%writefile services/haiti/main.py
import json

import services.common.tools as t


def search(args, adama):
    """Search database by building id."""

    building_id = args['building']
    for row in t.HAITI_DB:
        if row['Building'].startswith(building_id):
            print json.dumps(row, indent=4)
            print '---'
        

def list(args, adama):
    """List all rows."""

    for row in t.HAITI_DB:
        print json.dumps(row, indent=4)
        print '---'


## Interacting with Adama

To interact with Adama, we create an object with our credentials:

In [ ]:
import adamalib

TOKEN = ''                           # here goes your credentials
adama = adamalib.Adama('https://api.araport.org/community/v0.3',   # we use Araport for this example
                       token=TOKEN)                                

Now the `adama` object is connected to the server.  We can check that the server is up:

In [ ]:
adama.status

## Testing the new services locally

The services we are going to register in Adama can be tested first locally:

In [ ]:
import services.haiti.main

services.haiti.main.search({'building': 'A001'}, adama)

## Registering the services in Adama

Now we are ready to register the services in Adama.  We'll use the namespace `walter-dev` for testing purposes:

In [ ]:
adama['walter-dev'].services

To register a service, we just import its code and add it to the previous list (it may take a minute or two):

In [ ]:
import services.haiti.main            # the haiti service
import services.haiti_images.main     # the haiti_images service

haiti = adama['walter-dev'].services.add(services.haiti.main)
haiti_images = adama['walter-dev'].services.add(services.haiti_images.main)

haiti, haiti_images

The services are registered and can be accessed in https://araport.org.

But they can also be tested using the service object via Python.  Note that now the code will be executed remotely in the Adama server:

In [ ]:
haiti.search(building='A001')

In [ ]:
from IPython.display import Image

Image(data=haiti_images.search(building='A005', image=1).content)

In [ ]:
img = haiti_images.search(building='A001', image=1).content

In [ ]:
import base64
s = base64.encodestring(img)

In [ ]:
HTML('<img src="data:image/png;base64,{}" width=300/>'.format(s))

## Using the services

Now that the services are registered in Adama, and the Python objects `haiti` and `haiti_images` are connected to the **remote** services, we can use them as regular objects.

In [ ]:
!pip install pandas

In [ ]:
from pandas import DataFrame
x = DataFrame([[1,2], [2,3]], index=['Foo', 'Bar'], columns=['ds','sf'])
x

In [ ]:
haiti = adama['walter-dev'].haiti
haiti_images = adama['walter-dev'].haiti_images

Let's display the table of buildings together with their geographical coordinates:

In [94]:
full = haiti.list()
columns = ['Building', 'Latitude', 'Longitude']
buildings = DataFrame([[row[col] for col in columns] for row in full if row['Latitude']],
                      columns=columns)
buildings

,Building,Latitude,Longitude
0,A001,18.554472,-72.303361
1,A002,18.554361,-72.302972
2,A003(1),18.511667,-72.287389
3,A003(2),18.511667,-72.287389
4,A005,18.518111,-72.282222
5,A006,18.548639,-72.303194
6,A007,18.548917,-72.303861
7,A008,18.548806,-72.306833
8,A010,18.550944,-72.308250
9,A011,18.554083,-72.308556


In [ ]:
import jinja2
import functools32

In [46]:
from IPython.core.display import HTML, Javascript
from textwrap import dedent

def gmap_init():
    js = dedent(
        """
        window.gmap_initialize = function() {};
        $.getScript('https://maps.googleapis.com/maps/api/js?v=3&sensor=false&callback=gmap_initialize');
        """)
    return Javascript(data=js)

gmap_init()

<IPython.core.display.Javascript object>

In [47]:
%%html
<style type="text/css">
  .map-canvas { height: 400px; }
</style>

In [95]:
from IPython.core.display import HTML, Javascript
    
def map_display(data, 
                display=True, 
                center_latitude=18.5333, 
                center_longitude=-72.3333, 
                zoom=9):

    div_id = "map"
    html = '<div id="{}" class="map-canvas"/>'.format(div_id)
    marker_template = """
        var myLatlng = new google.maps.LatLng({lat},{lng});
        var marker_{i} = new google.maps.Marker({{ 
            position: myLatlng,
            map: map,
            title: "demo"
        }});
        var contentString = '';
        var infowindow_{i} = new google.maps.InfoWindow({{ 
            content: 'loading...'
        }});
        google.maps.event.addListener(marker_{i}, 'click', function() {{ 
            var xhr = new XMLHttpRequest();
            xhr.open('GET', 'https://api.araport.org/community/v0.3/walter-dev/haiti_images_v0.1/search?building={building}', true);
            xhr.responseType = 'blob';
            xhr.setRequestHeader('Authorization', 'Bearer {token}');

            xhr.onload = function(e) {{
                if (this.status == 200) {{
                    var blob = this.response;
                    var fr = new FileReader();
                    fr.onload = function() {{
                        var dataUrl = fr.result;
                        infowindow_{i}.setContent('<img src="' + dataUrl + '" width=300 />');
                    }};
                    fr.readAsDataURL(blob);
                }}
            }};
            xhr.send();
            infowindow_{i}.open(map, marker_{i});
            if (lastWindow) {{ 
                lastWindow.close();
            }}
            lastWindow = infowindow_{i};
      }});
    """
    js_init = """
        <script type="text/Javascript">
            (function() {{
                var mapOptions = {{
                    zoom: {zoom},
                    center: new google.maps.LatLng({lat}, {lng})
                }};
                var map = new google.maps.Map(document.getElementById('{elt}'), mapOptions);
                var lastWindow = false;
                var transitLayer = new google.maps.TransitLayer();
                transitLayer.setMap(map);
        """.format(zoom=zoom, lat=center_latitude, lng=center_longitude, elt=div_id)
    js_end = """
            })();  
        </script>
    """
    markers = [marker_template.format(i=i, lat=lat, lng=lng, token=TOKEN, building=bldg) 
               for (i, (bldg, lat, lng)) in enumerate(data)]
    js_markers = ''.join(markers)
    html = html + js_init + js_markers + js_end
    return HTML(html)

In [99]:
map_display(DataFrame.as_matrix(buildings))